In [1]:
# 安装相应的包
!pip install evaluate
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
# 导入相应的依赖
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import numpy as np

2025-06-05 01:18:27.588647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749086307.770027      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749086307.824135      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 加载数据集
orignal_data = load_dataset("doushabao4766/msra_ner_k_V3")

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [4]:
# 训练数据探索
print(orignal_data)
print(orignal_data['train'][18])
print("---------------------------")
print(orignal_data['train'][18]['id'])
print(orignal_data['train'][18]['tokens'])
print(orignal_data['train'][18]['ner_tags'])
print(orignal_data['train'][18]['knowledge'])

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})
{'id': '18', 'tokens': ['每', '当', '接', '到', '海', '外', '书', '友', '或', '归', '国', '人', '员', '汇', '至', '一', '册', '精', '美', '的', '食', '品', '图', '谱', '时', '，', '全', '家', '人', '欣', '喜', '若', '狂', '；', '而', '一', '册', '交', '换', '品', '离', '我', '而', '去', '，', '虽', '为', '复', '本', '，', '也', '大', '有', '李', '后', '主', '“', '挥', '泪', '别', '宫', '娥', '”', '之', '感', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': ''}
---------------------------
18
['每', '当', '接', '到', '海', '外', '书', '友', '或', '归', '国', '人', '员', '汇', '至', '一', '册', '精', '美', '的', '食', '品', '图', '谱', '时', '，', '全', '家', '人'

#### 通过数据结构看到实体已经被标记出来，根据数据集信息可知，整个数据集的实体标签 
class_label:    
    '0': O
    '1': B-PER
    '2': I-PER
    '3': B-ORG
    '4': I-ORG
    '5': B-LOC
    '6': I-LOC

In [5]:
#构造实体列表数据用于过滤不包含实体的数据
class_label_list = [1, 2, 3, 4, 5, 6]

# 实验 ner_tags 与 class_label_list 取交集
result = list(set(orignal_data['train'][18]['ner_tags']) & set(class_label_list))
print(result)
orignal_data = orignal_data.filter(lambda item: len(item['tokens']) <=512)
# 过滤掉源数据中ner_tags 不包含实体标注的数据
filter_data = orignal_data.filter(lambda item: len(list(set(item['ner_tags']) & set(class_label_list)))  > 0 & len(item['tokens']) <=512)

[1, 2]


Filter:   0%|          | 0/45001 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3443 [00:00<?, ? examples/s]

Filter:   0%|          | 0/44969 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3439 [00:00<?, ? examples/s]

In [6]:
len_check1 = orignal_data.filter(lambda item: len(item['tokens']) > 512)
print(len_check1)
len_check2 = filter_data.filter(lambda item: len(item['tokens']) > 512)
print(len_check2)

Filter:   0%|          | 0/44969 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3439 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 0
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 0
    })
})


Filter:   0%|          | 0/27526 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1971 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 0
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 0
    })
})


In [7]:
# 对比处理前和处理后的数据
print(orignal_data)
print("------------------------------")
print(filter_data)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 44969
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3439
    })
})
------------------------------
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 27526
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 1971
    })
})


In [8]:
# 初始化模型使用 模型名称为：google-bert/bert-base-chinese 最终预测类别为7个对应最后输出层神经元个数
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese", num_labels=7)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [10]:
# 实验使用分词器将tokens list 转换 input_data
print(filter_data['train'][0]['tokens'])
input_data = tokenizer(filter_data['train'][0]['tokens'], truncation=True, add_special_tokens = False, max_length = 512, is_split_into_words = True, return_tensors = 'pt')
input_data

['因', '有', '关', '日', '寇', '在', '京', '掠', '夺', '文', '物', '详', '情', '，', '藏', '界', '较', '为', '重', '视', '，', '也', '是', '我', '们', '收', '藏', '北', '京', '史', '料', '中', '的', '要', '件', '之', '一', '。']


{'input_ids': tensor([[1728, 3300, 1068, 3189, 2167, 1762,  776, 2966, 1932, 3152, 4289, 6422,
         2658, 8024, 5966, 4518, 6772,  711, 7028, 6228, 8024,  738, 3221, 2769,
          812, 3119, 5966, 1266,  776, 1380, 3160,  704, 4638, 6206,  816,  722,
          671,  511]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
# 数据进行处理将数据处理为能够数据模型的数据
def data_input_proc(item):
    # 输入文本转换模型输入token索引
    input_data = tokenizer(item['tokens'], truncation=True, add_special_tokens = False, padding = True, max_length = 512, is_split_into_words = True)
    # 数据是已经对齐的只需要将 item 中的 ner_tags 添加到 input data 中(不需要)
    input_data['labels'] = item['ner_tags']
    return input_data

In [12]:
# 处理数据
ds_orignal = orignal_data.map(data_input_proc, batched = True)
ds_filter = filter_data.map(data_input_proc, batched = True)

Map:   0%|          | 0/44969 [00:00<?, ? examples/s]

Map:   0%|          | 0/3439 [00:00<?, ? examples/s]

Map:   0%|          | 0/27526 [00:00<?, ? examples/s]

Map:   0%|          | 0/1971 [00:00<?, ? examples/s]

In [13]:
print(ds_orignal)
print("-------------------------")
print(ds_filter)

for item in ds_orignal['train']:
    print(item['tokens'])
    print(item['ner_tags'])
    print(item['input_ids'])
    print(item['ner_tags'])
    print(item['token_type_ids'])
    print(item['attention_mask'])
    print(item['labels'])
    break

print ("-------------------------------------")

for item in ds_filter['train']:
    print(item['tokens'])
    print(item['ner_tags'])
    print(item['input_ids'])
    print(item['ner_tags'])
    print(item['token_type_ids'])
    print(item['attention_mask'])
    print(item['labels'])
    break

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 44969
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3439
    })
})
-------------------------
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 27526
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1971
    })
})
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0

In [14]:
# 将数据集中需要输入模型的字段转换为张量
ds_orignal.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

ds_filter.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

#### 模型训练

In [15]:
# TrainingArguments
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 5,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

In [16]:
tags = ['0','B-PER','I-PER','B-ORG','I-ORG','B-LOC','I-LOC']
# 模型
id2lbl = {i:tag for i, tag in enumerate(tags)}
print(id2lbl)
lbl2id = {tag:i for i, tag in enumerate(tags)}
print(lbl2id)

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=7,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model

{0: '0', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'0': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [17]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts, labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

# 使用未经过滤处理的数据进行训练
# trainer1 = Trainer(
#     model,
#     args,
#     train_dataset=ds_orignal['train'],
#     eval_dataset=ds_orignal['test'],
#     data_collator=data_collator,
#     compute_metrics=compute_metric
# )

# 使用过滤处理的数据进行训练
trainer2 = Trainer(
    model,
    args,
    train_dataset=ds_filter['train'],
    eval_dataset=ds_filter['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [19]:
# 开始训练
trainer2.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,0.038787,"{'precision': 0.926829268292683, 'recall': 0.9172878949236777, 'f1': 0.9220338983050848, 'number': 2817}","{'precision': 0.8509861212563915, 'recall': 0.891354246365723, 'f1': 0.8707025411061285, 'number': 1307}","{'precision': 0.9603232916972814, 'recall': 0.9710252600297177, 'f1': 0.9656446250461767, 'number': 1346}","{'precision': 0.9157871591908531, 'recall': 0.9182098765432098, 'f1': 0.9169969176574195, 'number': 4536}",0.916054,0.921547,0.918792,0.988098
2,0.070800,0.040060,"{'precision': 0.955379632961497, 'recall': 0.9424920127795527, 'f1': 0.948892065761258, 'number': 2817}","{'precision': 0.8604982206405694, 'recall': 0.9250191277735271, 'f1': 0.8915929203539823, 'number': 1307}","{'precision': 0.9583333333333334, 'recall': 0.9569093610698366, 'f1': 0.9576208178438662, 'number': 1346}","{'precision': 0.9176981541802389, 'recall': 0.931657848324515, 'f1': 0.9246253145170112, 'number': 4536}",0.925491,0.937238,0.931327,0.989894
3,0.018400,0.042371,"{'precision': 0.9529791816223977, 'recall': 0.9424920127795527, 'f1': 0.9477065857576299, 'number': 2817}","{'precision': 0.8739255014326648, 'recall': 0.9334353481254782, 'f1': 0.9027007029226785, 'number': 1307}","{'precision': 0.9558498896247241, 'recall': 0.9650817236255572, 'f1': 0.9604436229205175, 'number': 1346}","{'precision': 0.9238302502720348, 'recall': 0.9358465608465608, 'f1': 0.9297995838352863, 'number': 4536}",0.929262,0.941335,0.935260,0.990391
4,0.008500,0.045972,"{'precision': 0.9573629523468291, 'recall': 0.9485268015619454, 'f1': 0.9529243937232525, 'number': 2817}","{'precision': 0.8852339181286549, 'recall': 0.9265493496557001, 'f1': 0.9054205607476635, 'number': 1307}","{'precision': 0.9587020648967551, 'recall': 0.9658246656760773, 'f1': 0.9622501850481125, 'number': 1346}","{'precision': 0.9308079702211517, 'recall': 0.9371693121693122, 'f1': 0.9339778095133473, 'number': 4536}",0.935727,0.942834,0.939267,0.990898
5,0.004500,0.049839,"{'precision': 0.9643500180050414, 'recall': 0.9506567270145545, 'f1': 0.9574544154451198, 'number': 2817}","{'precision': 0.8843636363636364, 'recall': 0.9303749043611323, 'f1': 0.906785980611484, 'number': 1307}","{'precision': 0.959499263622975, 'recall': 0.9680534918276374, 'f1': 0.9637573964497042, 'number': 1346}","{'precision': 0.9322256230870135, 'recall': 0.9400352733686067, 'f1': 0.9361141602634467, 'number': 4536}",0.938219,0.945533,0.941862,0.990836


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Trainer is attempting to log a value of "{'precision': 0.926829268292683, 'recall': 0.9172878949236777, 'f1': 0.9220338983050848, 'number': 2817}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8509861212563915, 'recall': 0.891354246365723, 'f1': 0.8707025411061285, 'number': 1307}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9603232916972814, 'recall': 0.9710252600297177, 'f1': 0.9656446250461767, 'number': 1346}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of T

TrainOutput(global_step=2155, training_loss=0.023946931356620346, metrics={'train_runtime': 6721.7418, 'train_samples_per_second': 20.475, 'train_steps_per_second': 0.321, 'total_flos': 3.541315169685031e+16, 'train_loss': 0.023946931356620346, 'epoch': 5.0})

#### 模型训练结束后的使用